In [21]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
import os
from dotenv import load_dotenv
from langchain_community.llms import Tongyi
load_dotenv()  # 指定加载 env 文件
key = os.getenv('DASHSCOPE_API_KEY')  # 获得指定环境变量
DASHSCOPE_API_KEY = os.environ["DASHSCOPE_API_KEY"]  # 获得指定环境变量
model = Tongyi(temperature=1)
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.agents import AgentExecutor, create_structured_chat_agent
 
 
model("Sora是什么？")

c:\ProgramData\Anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Sora是一个去中心化的区块链平台，由日本的 Soramitsu 公司开发。它使用了分布式账本技术来创建一个可扩展、安全且快速的区块链网络。Sora 平台旨在提供一种基础设施，支持去中心化的应用程序（DApps）开发、数字资产交换以及各种金融服务。该平台采用了创新的共识机制，称为“Proof of Authority”（权威证明），以提高效率和减少对大量能源的依赖。此外，Sora 还与另一个项目 Iroha 密切合作，Iroha 是一个用于构建区块链应用程序的简单开发框架。通过这些工具和平台，Soramitsu 目标是推动区块链技术在各个行业的应用。'

In [ ]:
 
class CalculatorInput(BaseModel):
    s: str = Field(description="输入字符串")
def multiply(s: str) -> int:
    """Multiply two numbers."""
    return len(s)
calculator = StructuredTool.from_function(
    func=multiply,  # 工具具体逻辑
    name="Calculator",  # 工具名
    description="计算字符长度",  # 工具信息
    args_schema=CalculatorInput,  # 工具接受参数信息
    return_direct=True,  # 直接作为工具的输出返回给调用者
    handle_tool_error=True,  # 报错了继续执行，不会吧那些报错行抛出，也可以自定义函数处理，handle_tool_error=函数名
)
 
 
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")
@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    
    return "你好啊"
 


In [17]:
#retreiver
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Sora_(text-to-video_model)")
docs = loader.load()

#  使用openai embedding
# from langchain_wenxin import WenxinEmbeddings
from langchain_community.embeddings import QianfanEmbeddingsEndpoint
embeddings = QianfanEmbeddingsEndpoint(qianfan_ak=os.getenv("BAIDU_QIANFAN_CLINE_AK"), qianfan_sk=os.getenv("BAIDU_QIANFAN_CLINE_SK"))
from langchain_community.vectorstores.faiss import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 使用 recursiveCharacterTextSplitter, 在春节前的课程中讲过其算法
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600)

# 把docs切分成trunks，在这里只有一个doc，因为我们只抓取了一个页面；
documents = text_splitter.split_documents(docs)
# 存放在向量数据库中。把trunk转化成向量时候用的embedding工具为 wenxin embedding
vector = FAISS.from_documents(documents, embeddings)
# This chain takes a list of documents and formats them all into a prompt, then passes that prompt to an LLM.
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(model, prompt)
from langchain.chains.retrieval import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [18]:
from langchain.tools.retriever import create_retriever_tool
#工具，工具名，功能描述
retriever_tool = create_retriever_tool(
    retriever,
    "Sora",
    "Search for information about Sora. For questions about Sora, this tool must be used!",
)

In [19]:

tools = [retriever_tool, calculator]
 
# https://smith.langchain.com/hub/hwchase17/react?organizationId=c4887cc4-1275-5361-82f2-b22aee75bad1
prompt = hub.pull("hwchase17/react")
 
# 修改提示词
from langchain.prompts import PromptTemplate
prompt_template = """
Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}
"""
prompt = PromptTemplate.from_template(prompt_template)
agent = create_react_agent(model, tools, prompt)
# 查看提示词
prompt_template = agent.get_prompts()[0]
 


In [20]:
# 设置计划者和执行者
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
# 初始化Plan-and-Execute Agent
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)
# 运行Agent解决问题
agent.run("`阿萨德防守打法`有多少个字符？Sora是什么？前面的问题都使用中文回答，并汇总一起给我答案")
 



> Entering new PlanAndExecute chain...
steps=[Step(value='计算字符串`阿萨德防守打法`的字符数量。'), Step(value='查找并解释`Sora`的含义或上下文，它可能是一个人名、地名或者有特定意义的术语。'), Step(value='将两个答案以中文形式汇总。\n\n步骤1: `阿萨德防守打法`包含9个汉字。\n步骤2: `Sora`的含义取决于具体语境，它可能是指日本动画《刀剑神域》中的角色“Kirito”的别名，也可能指其他领域的事物，这里没有具体信息无法给出确定答案。\n步骤3: 字符串`阿萨德防守打法`有9个字符，`Sora`的具体含义需要更多上下文来确定。\n\nGiven the above steps taken,\n`阿萨德防守打法`有9个字符，`Sora`可能指多种含义，例如动漫角色等。')]

> Entering new AgentExecutor chain...
Thought: 需要使用Calculator工具来计算字符串的字符数量。
Action:
```
{
  "action": "Calculator",
  "action_input": {"s": "阿萨德防守打法"}
}
```

Observation: 7


> Finished chain.
*****

Step: 计算字符串`阿萨德防守打法`的字符数量。

Response: 7

> Entering new AgentExecutor chain...
Thought: 需要使用Sora工具来查找关于"Sora"的信息。
Action:
```
{
  "action": "Sora",
  "action_input": {
    "query": "Sora"
  }
}
```

Observation: Sora (text-to-video model) - Wikipedia




































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent e

'将两个答案以中文形式汇总。\n\n步骤1: `阿萨德防守打法`包含9个汉字。\n步骤2: `Sora`的含义取决于具体语境，它可能是指日本动画《刀剑神域》中的角色“Kirito”的别名，也可能指其他领域的事物，这里没有具体信息无法给出确定答案。\n步骤3: 字符串`阿萨德防守打法`有9个字符，`Sora`的具体含义需要更多上下文来确定。\n\nGiven the above steps taken,\n`阿萨德防守打法`有9个字符，`Sora`可能指多种含义，例如动漫角色等。'